# Helpful (hopefully) notes and general measurement tips

- **Spectrum Analyzer:** The spectrum analyzer tends to timeout if it hasn't been used in a while. To restart it, simply execute the qubit initialization again or use the qb._instrument.set() command to send a value and wake the instrument back up.
- 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qubit import qubit
import numpy as np
import plot_functions as pf
#from instrument_init import init_sa

2024-04-25 13:06:10,779 - qm - INFO     - Starting session: 108a5227-1e44-4d60-a93a-d4f47b6e8f69


# Initialize qubit

In [ ]:
qb = qubit('ocs2')

# Time of flight

For tof calibration, we don't need the resontor frequency per se

In [ ]:
qb.update_value('readout_atten',0)
qb.update_value('tof',56) #clock cycles
qb.update_value('smearing',10)
qb.update_value('n_avg',1000)
qb.update_value('rr_IF',5e6) 
qb.update_value('readout_length',1000)

adc1,adc2 = qb.tof_cal(update_tof=True)
pf.tof_plot(adc1, adc2,delay=qb.pars['tof'],offsets=qb.pars['analog_input_offsets'])

In [ ]:
adc1,adc2 = qb.tof_cal(update_tof=True)
pf.tof_plot(adc1, adc2,delay=qb.pars['tof'],offsets=qb.pars['analog_input_offsets'])

**Note:** execute the following cell to visually see if the tof is calibrated

In [ ]:
adc1,adc2 = qb.tof_cal(update_tof=False)
pf.tof_plot(adc1, adc2,delay=qb.pars['tof'],offsets=qb.pars['analog_input_offsets'])

# Optimize Mixers

Starting with the smallest resonator

In [ ]:
LO_freq = 7.5e9 #Hz (targets the smallest resonator)

In [ ]:
# qb.update_value('rr_freq',7.6146e9)
qb.pars['rr_IF']

## Minimize LO leakage 

## Resonator

In [ ]:
# qb.update_value('rr_LO',LO_freq)
qb.update_value('rr_LO',qb.pars['rr_freq']-qb.pars['rr_IF'])
# qb.get_power(fc=qb.pars['rr_LO'],threshold=-20,span=1e6,config=True,plot=True)
qb.opt_lo_leakage(mode='coarse',element='rr',sa_span=0.5e6,threshold=-20,plot=True)

## Minimize image sideband

In [ ]:
qb.update_value('rr_IF',50e6)
qb.opt_sideband(mode='coarse',element='rr',sa_span=0.5e6,threshold=-20,plot=True)

In [ ]:
qb.opt_sideband(mode='coarse',element='rr',sa_span=0.5e6,threshold=-50,plot=True)

# Resonator spectroscopy

**Notes on resonator spectroscopy**

In [ ]:
IF_min, IF_max = 36e6, 41.5e6
df_resonator_spec = 0.05e6

In [ ]:
job.execution_report()

In [ ]:
qb.update_value('readout_length', 5000)   #ns (since resonator is narrow)
qb.pars['resettime']['rr'] = 5000 # clock cycles
qb.write_pars()
qb.update_value('n_avg',3000)
qb.update_value('readout_atten',10)
df_resonator_spec = 0.05e6
qb.experiment = 'resonator_spec'
##added a 10db fixed attenuator on rr line after the spiltter and just before fridge input
data, job = qb.resonator_spec(
                       f_LO = 7.8e9,
                       IF_min = IF_min,
                       IF_max = IF_max,
                       df = df_resonator_spec,
                       fit=True,
                       on_off=False,
                       savedata=True,
                       showprogress=True)

In [ ]:
fc,fwhm = pf.fit_res(data['freqs'],data['I'],data['Q'])
pf.resonator_spec_plot(data,qb_pars=qb.pars,fwhm=fwhm,fc=fc)

In [ ]:
from resonator import Resonator

In [ ]:
res = Resonator(port_type='n',f_data=data['freqs'],z_data=data['I']+1j*data['Q'])
res.autofit(electric_delay=0)
res.show()

# Qubit Punchout

In [ ]:
attenuations = np.arange(0,50,5)
qb.update_value('n_avg',2000)
data, job = qb.punchout(df = 0.05e6,
                 IF_min = IF_min,
                 IF_max = IF_max,
                 attenuations=attenuations,
                 savedata=False)

In [ ]:
pf.punchout_plot(data,xlabel='Frequency (GHz)',ylabel='Attenuation (dB)',cbar_label='Phase (deg)')

In [ ]:
qb.update_value('rr_atte')

# Qubit Spectroscopy

The first time we run qubit spectroscopy we have to use a high qubit driving amplitude for a couple of reasons. 
1. Detect higher levels like $\omega_{02}/2$ and $\omega_{03}/3$ so you can calculate anharmonicity.
2. Since you don't know exactly what the qubit frequency is going to be (unless you've nailed the fab process), you will have to execute a broad sweep. Driving the qubit at high power broadens the resonance (decreases coherence), makes it easier to detect $\omega_{01}$ -> frequency spacing can be larger.

## Narrow search

In [ ]:
qb.pars['qubit_IF']

In [ ]:
qb.update_value('qubit_LO',6e9)
qb.get_power(fc=qb.pars['qubit_LO'],threshold=-20,sa_span=1e6,config=True,plot=True)   

In [ ]:
amp_q_scaling = 1
qb.update_value('n_avg',5000)
qb.update_value('rr_freq', 7.61805e9)
# qb.update_value('rr_IF', 50e6)
# qb.update_value('smearing',40)
# qb.update_value('tof',292)
qb.update_value('readout_atten',27)
qb.update_value('rr_LO', qb.pars['rr_freq'] - qb.pars['rr_IF'])
qb.pars['resettime']['qubit'] = int(20e3)
qb.write_pars()
qb.experiment = 'qubit_spec'
data, job = qb.qubit_spec(element='qubit',
                                check_mixers=True,
                                IF_min=50e6,
                                IF_max=200e6,
                                f_LO=6.6e9,
                                df=0.5e6,
                                on_off=True,
                                amp_q_scaling = amp_q_scaling,
                                saturation_dur = 20e3,
                                showprogress=True,
                                savedata=False)

In [ ]:
pf.qubit_spec_plot(data,qb_pars=qb.pars,find_peaks=True, amp_q_scaling=amp_q_scaling)

## Run broad spectroscopy scan on qubit

In [ ]:
qb._instruments.set('sa','frequency',6e9)
qb.pars['resettime']['qubit'] = int(5e3)

In [ ]:
qb.update_value('n_avg',2000)
qb.update_value('rr_freq', 7.61805e9)
qb.update_value('rr_IF', 50e6)
qb.update_value('rr_LO', qb.pars['rr_freq'] - qb.pars['rr_IF'])
qb.pars['resettime']['qubit'] = int(10e3)
qb.write_pars()
df_qubit_spec = 2e6
data, job = qb.run_scan(df = df_qubit_spec, element='qubit',
                                check_mixers=False,
                                chunksize = 200e6,
                                lo_min = 6e9,
                                lo_max = 7e9,
                                amp_q_scaling = 1,
                                on_off=True,
                                saturation_dur = 20e3,                   
                                showprogress=True,
                                savedata=False)

In [ ]:
%matplotlib inline
pf.qubit_spec_plot(data,qb_pars=qb.pars,df=df_qubit_spec,rrFreq=qb.pars['rr_freq'],amp_q_scaling=1)